In [1]:
from aut import *
from pyspark.sql.functions import col, desc

In [2]:
# Web archive collection; web pages.
webpages = WebArchive(sc, sqlContext, "/home/nruest/Projects/au/sample-data/geocities").webpages()

In [3]:
# Web archive collection; web graph.
webgraph = WebArchive(sc, sqlContext, "/home/nruest/Projects/au/sample-data/geocities").webgraph()

In [4]:
# Domains file.
webpages.groupBy("domain")\
  .count()\
  .sort(col("count").desc())\
  .write.csv("all-domains")

[2022-05-22T16:56:13.582Z - 00007 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143451-00102-ia400108.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-05-22T16:56:13.582Z - 00005 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143340-00105-ia400105.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-05-22T16:56:13.582Z - 00008 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142731-00177-ia400130.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-05-22T16:56:13.583Z - 00002 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143856-00108-ia400107.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-05-22T16:56:13

22/05/22 12:56:14 WARN PDFParser: J2KImageReader not loaded. JPEG2000 files will not be processed.
See https://pdfbox.apache.org/2.0/dependencies.html#jai-image-io
for optional dependencies.

22/05/22 12:56:14 WARN TesseractOCRParser: Tesseract OCR is installed and will be automatically applied to image files unless
you've excluded the TesseractOCRParser from the default parser.
Tesseract may dramatically slow down content extraction (TIKA-2359).
As of Tika 1.15 (and prior versions), Tesseract is automatically called.
In future versions of Tika, users may need to turn the TesseractOCRParser on via TikaConfig.
22/05/22 12:56:14 WARN SQLite3Parser: org.xerial's sqlite-jdbc is not loaded.
Please provide the jar on your classpath to parse sqlite files.
See tika-parsers/pom.xml for the correct version.


In [5]:
# Full-text.
webpages.write.csv("full-text")

[2022-05-22T17:00:51.618Z - 00435 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142649-00105-ia400111.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-05-22T17:00:51.618Z - 00438 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143451-00102-ia400108.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-05-22T17:00:51.621Z - 00439 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027142731-00177-ia400130.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-05-22T17:00:51.623Z - 00434 - HdfsIO] Opening file file:/home/nruest/Projects/au/sample-data/geocities/GEOCITIES-20091027143351-00117-ia400103.us.archive.org.warc.gz (Offset: 0, length: 0, decompress: false, strategy: BlockWise [dynamic])
[2022-05-22T17:00:51

In [6]:
# Create DataFrame for GraphML output
graph = webgraph.groupBy("crawl_date", remove_prefix_www(extract_domain("src")).alias("src_domain"), remove_prefix_www(extract_domain("dest")).alias("dest_domain"))\
          .count()\
          .filter((col("dest_domain").isNotNull()) & (col("dest_domain") !=""))\
          .filter((col("src_domain").isNotNull()) & (col("src_domain") !=""))\
          .filter(col("count") > 5)\
          .orderBy(desc("count"))

In [8]:
# Write the GraphML out to a file.
WriteGraphML(graph.collect(), "test.graphml")

In [9]:
# Write the GEXF out to a file.

# Not part of auk standard derivative process,
# but apart of https://github.com/archivesunleashed/aut/pull/466 testing.

WriteGEXF(graph.collect(), "test.gexf")